In [ ]:
%%capture
!pip install sahi ultralytics


In [ ]:
TEST_FILES=['1772.tif', '1181.tif', '105.tif', '2493.tif', '2532.tif', '1703.tif', '1224.tif', '2207.tif', '128.tif', '415.tif', '307.tif', '285.tif', '140.tif', '18.tif', '1118.tif', '142.tif', '1973.tif', '2475.tif', '1815.tif', '2515.tif', '615.tif', '2436.tif', '2584.tif', '2118.tif', '2316.tif', '1799.tif', '2531.tif', '303.tif', '462.tif', '1922.tif', '10.tif', '1425.tif', '379.tif', '766.tif', '931.tif', '2318.tif', '1462.tif', '2106.tif', '1920.tif', '2561.tif', '1579.tif', '2571.tif', '1633.tif', '650.tif', '574.tif', '1701.tif', '345.tif', '1927.tif', '1052.tif', '2490.tif', '1828.tif', '418.tif', '2495.tif', '2072.tif', '2044.tif', '740.tif', '107.tif', '2053.tif', '1818.tif', '2370.tif', '473.tif', '1505.tif', '914.tif', '2130.tif', '2148.tif', '1740.tif', '1985.tif', '399.tif', '1881.tif', '2454.tif', '1372.tif', '1351.tif', '540.tif', '2032.tif', '1436.tif', '1897.tif', '104.tif', '2484.tif', '2021.tif', '1607.tif', '2505.tif', '670.tif', '5.tif', '1441.tif']

In [ ]:
""" USO SOLO LE IMMAGINI DI TEST """
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction
from sahi.utils.cv import read_image
from IPython.display import Image
import os, random

# 1. Carica il tuo modello YOLO11 personalizzato
detection_model = AutoDetectionModel.from_pretrained(
    model_type="ultralytics",
    model_path="/kaggle/input/actual-yolo11-xview-best/pytorch/default/1/actual_best.pt",  # any yolov8/yolov9/yolo11/yolo12/rt-detr det model is supported
    confidence_threshold=0.25,
    device="cuda:0",  # or 'cuda:0' if GPU is available
)

# 2. Carica l'immagine grande di xView
image_folder = "/kaggle/input/xview-dataset/train_images/train_images"
supported_extensions = ('.tif', '.tiff', '.jpg', '.jpeg', '.png')
all_images = [f for f in os.listdir(image_folder) if f.lower().endswith(supported_extensions)]
if not all_images:
    raise FileNotFoundError(f"Nessuna immagine trovata in {image_folder}")


# Verifica quali immagini della tua lista esistono effettivamente nella cartella
valid_test_images = []

for filename in TEST_FILES:
    # Se nella lista hai messo solo numeri (es: "10"), aggiungiamo .tif
    # Se hai già messo l'estensione, questo blocco viene saltato.
    if not filename.lower().endswith(('.tif', '.tiff', '.jpg', '.jpeg', '.png')):
        filename = f"{filename}.tif"
    
    full_path = os.path.join(image_folder, filename)
    
    # Se il file esiste fisicamente nella cartella, lo aggiungiamo ai candidati
    if os.path.exists(full_path):
        valid_test_images.append(filename)

# Controllo sicurezza
if not valid_test_images:
    raise FileNotFoundError("Nessuna delle immagini presenti in TEST_FILES è stata trovata nella cartella specificata.")

print(f"Trovate {len(valid_test_images)} immagini valide dalla lista di test.")

random_image_name = random.choice(valid_test_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

In [ ]:
random_image_name = random.choice(valid_test_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

In [ ]:
random_image_name = random.choice(valid_test_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

In [ ]:
random_image_name = random.choice(valid_test_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

In [ ]:
random_image_name = random.choice(valid_test_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

In [ ]:
random_image_name = random.choice(valid_test_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

In [ ]:
random_image_name = random.choice(valid_test_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

In [ ]:
random_image_name = random.choice(valid_test_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

In [ ]:
random_image_name = random.choice(valid_test_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

In [ ]:
random_image_name = random.choice(valid_test_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png

In [ ]:
random_image_name = random.choice(valid_test_images)
random_image_path = os.path.join(image_folder, random_image_name)
print(f"Immagine selezionata per SAHI: {random_image_name}")

#image = read_image("large_xview_image.tif")

# 3. Esegui l'inferenza a fette (Slicing)
result = get_sliced_prediction(
    random_image_path,
    detection_model,
    slice_height=640,        # Dimensione della fetta (coerente con il tuo training)
    slice_width=640,
    overlap_height_ratio=0.2, # Sovrapposizione per evitare di tagliare oggetti a metà
    overlap_width_ratio=0.2
)


# 4. Visualizza o salva i risultati
result.export_visuals(
    export_dir="results_sahi/" + str(random_image_name),
    file_name="prediction_visual",
    rect_th=1,       # Spessore del rettangolo (bounding box): 1 pixel
    text_size=0.3,   # Dimensione del testo: prova 0.3 o 0.4
    #text_th=1        # Spessore del testo: 1 pixel
)
# Separa il nome dall'estensione (es. '2279.tif' -> '2279')
image_name_only = os.path.splitext(random_image_name)[0]
Image('/kaggle/working/results_sahi/'+str(image_name_only)+'.tif/prediction_visual.png',width=2048) #/kaggle/working/results_sahi/310.tif/prediction_visual.png